# Settings

In [47]:
data_filepath = "PaintShop - September 2024.xlsx"
data_sheet_names_by_table_name = {
    "orders": "Orders", 
    "machines": "Machines", 
    "setups": "Setups"
}  

# Data

In [48]:
import pandas as pd

In [49]:
data = {table_name: pd.read_excel(data_filepath, sheet_name) for table_name, sheet_name in data_sheet_names_by_table_name.items()}

orders = pd.read_excel(data_filepath, sheet_name = "Orders")
machines = pd.read_excel(data_filepath, sheet_name = "Machines")
setups = pd.read_excel(data_filepath, sheet_name = "Setups")

In [50]:
data["orders"]

,Order,Surface,Colour,Deadline,Penalty
0,O1,150,Green,18,10
1,O2,200,Yellow,28,12
2,O3,180,Blue,12,8
3,O4,170,Blue,15,15
4,O5,130,Yellow,36,9
5,O6,160,Blue,33,11
6,O7,140,Green,35,14
7,O8,210,Yellow,38,13
8,O9,190,Blue,32,7
9,O10,120,Green,41,10


In [51]:
data["machines"].head()

,Machine,Speed
0,M1,20
1,M2,25
2,M3,18


In [52]:
data["setups"]

,From colour,To colour,Setup time
0,Green,Red,6
1,Yellow,Green,2
2,Green,Yellow,5
3,Yellow,Red,1
4,Red,Yellow,8
5,Red,Green,5
6,Blue,Red,8
7,Blue,Yellow,10
8,Blue,Green,4
9,Green,Blue,3


### Prepare data

In [53]:
# Encode color names for efficiency, set ID as index
unique_colors = data["setups"]["From colour"].unique()
color_names_by_index = pd.DataFrame({
    "name": [c_name for c_name in unique_colors]
}, index = range(len(unique_colors)))
color_names_by_index

,name
0,Green
1,Yellow
2,Red
3,Blue


In [54]:
# Transpose setups table, encode colors and set color ID to index
setups = pd.DataFrame({ 
    color_names_by_index[color_names_by_index["name"] == c1].index[0]: [
        data["setups"][
            (data["setups"]["To colour"] == c1) & (data["setups"]["From colour"] == c2)
        ]["Setup time"].values[0] if c1 != c2 else 0 for c2 in color_names_by_index["name"]
    ] for c1 in color_names_by_index["name"] }, index = color_names_by_index.index)
setups

,0,1,2,3
0,0,5,6,3
1,2,0,1,11
2,5,8,0,4
3,4,10,8,0


In [64]:
# Fix orders table: encode colors and set machine ID as index
orders = pd.DataFrame(
    {
        "surface": data["orders"]["Surface"].values,
        "color": [color_names_by_index[color_names_by_index["name"] == c].index[0] for c in data["orders"]["Colour"]],
        "deadline": data["orders"]["Deadline"].values,
        "penalty": data["orders"]["Penalty"].values,
    },
    index = [order_name[1:] for order_name in data["orders"]["Order"].values]
)
orders  

,surface,color,deadline,penalty
1,150,0,18,10
2,200,1,28,12
3,180,3,12,8
4,170,3,15,15
5,130,1,36,9
6,160,3,33,11
7,140,0,35,14
8,210,1,38,13
9,190,3,32,7
10,120,0,41,10


In [73]:
data["machines"]
machines = pd.DataFrame({
    "speed": data["machines"]["Speed"].values
}, index = [m_name[1:] for m_name in data["machines"]["Machine"]])
machines

,speed
1,20
2,25
3,18
